<a href="https://colab.research.google.com/github/eduseiti/ia368v_dd_final/blob/master/colbertx_fine_tune_48_000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prepare the environment

In [1]:
import os
import sys

In [2]:
IN_COLAB='google.colab' in sys.modules
LINK_WITH_COMET=False

In [3]:
if IN_COLAB:
    from google.colab import drive

    WORKING_FOLDER="/content/drive/MyDrive/unicamp/ia368v_dd/trabalho_final_UNICAMP-IR"

    drive.mount('/content/drive', force_remount=True)

    os.chdir(WORKING_FOLDER)
    
    !pip install transformers -q
    !pip install mlflow -q
    !pip install ujson -q
    !pip install faiss-gpu -q
    !pip install SentencePiece -q
    
    if not os.path.exists("ColBERT-X"):
        !git clone https://github.com/hltcoe/ColBERT-X.git
            
    PYTHON="python3"
else:
    WORKING_FOLDER="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/trabalho_final/"
    
    PỲTHON="python3.8"

Mounted at /content/drive


In [4]:
!nvidia-smi

Sun Jun 11 21:37:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    50W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
COLBERT_EXPERIMENTS_FOLDER="colbertx_experiments"

In [6]:
if not os.path.exists(COLBERT_EXPERIMENTS_FOLDER):
    os.makedirs(COLBERT_EXPERIMENTS_FOLDER)

In [7]:
os.chdir("ColBERT-X")

In [8]:
BATCH_SIZE=12

In [9]:
NUM_OF_STEPS=int(1e+6 // BATCH_SIZE + 1)

In [10]:
NUM_OF_STEPS

83334

In [11]:
TRIPLETS_SUFFIX="000"

In [12]:
TRIPLETS_FILE="mMARCO_triplets_{}.tsv".format(TRIPLETS_SUFFIX)
FINE_TUNE_FILE="fine_tune_{}".format(TRIPLETS_SUFFIX)

In [ ]:
MODEL_CHECKPOINT="../colbertx_experiments/mMSMARCO-pt_048_dim/train.py/fine_tune_000/checkpoints/colbert-20000.dnn"

### Fine-tune from original model

In [16]:
!{PYTHON} -m xlmr_colbert.train --amp --doc_maxlen 480 --bsize {BATCH_SIZE} --accum 1 \
--triples ../{TRIPLETS_FILE} --maxsteps 20000 \
--root ../{COLBERT_EXPERIMENTS_FOLDER}/ --experiment mMSMARCO-pt_048_dim --dim 48 --similarity l2 --run {FINE_TUNE_FILE}

Streaming output truncated to the last 5000 lines.
[Jun 11, 13:00:23] 17501 0.21193383871113536
#>>>    -11.71 -24.19 		|		 12.48
[Jun 11, 13:00:24] 17502 0.21192176766310725
#>>>    -13.13 -22.81 		|		 9.679999999999998
[Jun 11, 13:00:24] 17503 0.21191366595132483
#>>>    -11.42 -24.04 		|		 12.62
[Jun 11, 13:00:24] 17504 0.21190157948218508
#>>>    -13.22 -21.89 		|		 8.67
[Jun 11, 13:00:24] 17505 0.2118923400638082
#>>>    -12.95 -22.05 		|		 9.100000000000001
[Jun 11, 13:00:25] 17506 0.21188211930101425
#>>>    -12.28 -24.73 		|		 12.450000000000001
[Jun 11, 13:00:25] 17507 0.21187013073200434
#>>>    -15.08 -23.13 		|		 8.049999999999999
[Jun 11, 13:00:25] 17508 0.21186512483435632
#>>>    -12.4 -21.39 		|		 8.99
[Jun 11, 13:00:26] 17509 0.2118541238499063
#>>>    -15.14 -23.56 		|		 8.419999999999998
[Jun 11, 13:00:26] 17510 0.21184570901365782
#>>>    -14.96 -22.6 		|		 7.640000000000001
[Jun 11, 13:00:26] 17511 0.21183700281654144
#>>>    -13.13 -23.57 		|		 10.44
[Jun 11, 13:0

### Fine-tune from checkpoint

In [17]:
TRIPLETS_SEQUENCE=["029", "008", "017", "006", "012", "025"]
previous_triplet="000"

In [18]:
for which_triplet in TRIPLETS_SEQUENCE:

    triplets_file="mMARCO_triplets_{}.tsv".format(which_triplet)
    fine_tune_file="fine_tune_{}".format(which_triplet)
    model_checkpoint="../colbertx_experiments/mMSMARCO-pt_048_dim/train.py/fine_tune_{}/checkpoints/colbert-20000.dnn".format(previous_triplet)

    !{PYTHON} -m xlmr_colbert.train --amp --doc_maxlen 480 --bsize {BATCH_SIZE} --accum 1 \
        --checkpoint {model_checkpoint} --triples ../{triplets_file} --maxsteps 20000 \
        --root ../{COLBERT_EXPERIMENTS_FOLDER}/ --experiment mMSMARCO-pt_048_dim --dim 48 --similarity l2 --run {fine_tune_file}

    previous_triplet = which_triplet

Streaming output truncated to the last 5000 lines.
[Jun 11, 21:19:23] 19618 0.0793378484671202
#>>>    -12.15 -24.22 		|		 12.069999999999999
[Jun 11, 21:19:23] 19619 0.07933405223967939
#>>>    -10.82 -23.4 		|		 12.579999999999998
[Jun 11, 21:19:24] 19620 0.07933002156120496
#>>>    -11.41 -23.39 		|		 11.98
[Jun 11, 21:19:24] 19621 0.07934730298231778
#>>>    -11.89 -23.36 		|		 11.469999999999999
[Jun 11, 21:19:24] 19622 0.0793456784333071
#>>>    -11.61 -24.68 		|		 13.07
[Jun 11, 21:19:25] 19623 0.07934211955363375
#>>>    -14.62 -24.36 		|		 9.74
[Jun 11, 21:19:25] 19624 0.07933871754681925
#>>>    -9.98 -22.4 		|		 12.419999999999998
[Jun 11, 21:19:25] 19625 0.07933508595170534
#>>>    -9.59 -24.08 		|		 14.489999999999998
[Jun 11, 21:19:25] 19626 0.07933198762585197
#>>>    -12.08 -24.72 		|		 12.639999999999999
[Jun 11, 21:19:26] 19627 0.07933546466349699
#>>>    -10.82 -25.09 		|		 14.27
[Jun 11, 21:19:26] 19628 0.0793320065717501
#>>>    -9.11 -22.18 		|		 13.07
[Jun 11, 21

In [11]:
def fine_tune_colbert(current_triplet, previous_triplet):
    triplets_file="mMARCO_triplets_{}.tsv".format(current_triplet)
    fine_tune_file="fine_tune_{}".format(current_triplet)
    model_checkpoint="../colbertx_experiments/mMSMARCO-pt_048_dim/train.py/fine_tune_{}/checkpoints/colbert-20000.dnn".format(previous_triplet)

    !{PYTHON} -m xlmr_colbert.train --amp --doc_maxlen 480 --bsize {BATCH_SIZE} --accum 1 \
        --checkpoint {model_checkpoint} --triples ../{triplets_file} --maxsteps 20000 \
        --root ../{COLBERT_EXPERIMENTS_FOLDER}/ --experiment mMSMARCO-pt_048_dim --dim 48 --similarity l2 --run {fine_tune_file}

In [13]:
fine_tune_colbert("025", "012")

Streaming output truncated to the last 5000 lines.
[Jun 11, 23:03:44] 17501 0.07876782052140464
#>>>    -10.95 -22.58 		|		 11.629999999999999
[Jun 11, 23:03:44] 17502 0.07876590749793072
#>>>    -12.1 -22.33 		|		 10.229999999999999
[Jun 11, 23:03:45] 17503 0.0787617606283428
#>>>    -10.43 -24.3 		|		 13.870000000000001
[Jun 11, 23:03:45] 17504 0.07875745632723054
#>>>    -12.25 -23.91 		|		 11.66
[Jun 11, 23:03:45] 17505 0.07876704353105353
#>>>    -10.52 -25.15 		|		 14.629999999999999
[Jun 11, 23:03:46] 17506 0.07876300579950697
#>>>    -10.34 -21.61 		|		 11.27
[Jun 11, 23:03:46] 17507 0.07875858439444886
#>>>    -12.12 -22.03 		|		 9.910000000000002
[Jun 11, 23:03:46] 17508 0.0787589115632947
#>>>    -9.82 -20.95 		|		 11.129999999999999
[Jun 11, 23:03:46] 17509 0.07875601820588105
#>>>    -11.01 -23.13 		|		 12.12
[Jun 11, 23:03:47] 17510 0.07875176609448703
#>>>    -12.12 -23.64 		|		 11.520000000000001
[Jun 11, 23:03:47] 17511 0.07874947310741838
#>>>    -9.94 -22.5 		|		 12.